In [2]:
from openeye.oechem import *
from openeye.oeomega import *
import pandas as pd

csv = "SAMPL8_molecule_ID_and_SMILES.csv"
redo = []

with open(csv) as file:
    #next(file)
    for line in file:
        #initialize omega, this is a conformation generator
        omega = OEOmega()
        #set the maximum conformer generated to 1
        omega.SetMaxConfs(1) 
        omega.SetIncludeInput(False)
        #Leniency in assigning atom types
        omega.SetStrictAtomTypes(False) 
        # Don't generate conformers if stereochemistry not provided
        # Setting to false would pick a random stereoisomer
        omega.SetStrictStereo(True) 

        # Generate OEMol 
        mol = OEMol()
        name = line.split(',')[0]
        smiles = line.split(',')[1].strip()
        OEParseSmiles(mol, smiles)
        #print("canonical",OECreateCanSmiString(mol))

        #Parse the SMILES string
        OEParseSmiles(mol, smiles)
        status = omega(mol)
        if not status:
            print("Error generating conformers for %s, %s." % (smiles, name))
            redo.append([name,smiles])
        
        # Write out PDB of molecule
        ofile = oemolostream('%s.pdb'%(name))
        OEWriteMolecule(ofile, mol)
        ofile.close()
        
        # Write out MOL2 of molecule
        ofile = oemolostream('%s.mol2'%(name))
        OEWriteMolecule(ofile, mol)
        ofile.close()

        # Write out SDF of molecule
        ofile = oemolostream('%s.sdf'%(name))
        OEWriteMolecule(ofile, mol)
        ofile.close()